<a href="https://colab.research.google.com/github/YvixMiHo/TensorFlow_Developer_Skills/blob/main/MultiClassclassification_TensorFlow_LearningObjectives_2_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal use multiclass classification to analyze a CSV file

# 1. Import Depenedencies

In [ ]:
import csv
import string
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from os import getcwd

# 2. Load Enviornment / Data set

In [ ]:
# sign_mnist_train.csv
!gdown --id 1z0DkA9BytlLxO1C0BAWzknLyQmZAp0HR
# sign_mnist_test.csv
!gdown --id 1z1BIj4qmri59GWBG4ivMNFtpZ4AXIbzg

## 2.a Make sense of the data, use confusion matrix if necessary

In [ ]:
with open(TRAINING_FILE) as training_file:
  line = training_file.readline()
  print(f"First line (header) looks like this:\n{line}")
  line = training_file.readline()
  print(f"Each subsequent line (data points) look like this:\n{line}")

In [ ]:
from matplotlib import image
# GRADED FUNCTION: parse_data_from_input
def parse_data_from_input(filename):

  with open(filename) as file:
      csv_reader = csv.reader(file, delimiter=',')
      
      labels = []
      images = []
      
      next(csv_reader,None)

      for row in csv_reader:
          label = row[0]
          image_sample = row[1:]

          image_array = np.array(image_sample).reshape((28,28))
      
          labels.append(label)
          images.append(image_array)

      labels = np.array(labels).astype(float)
      images = np.array(images).astype(float)

  return images, labels

## 2.b Validate data shape*

In [ ]:
training_images, training_labels = parse_data_from_input(TRAINING_FILE)
validation_images, validation_labels = parse_data_from_input(VALIDATION_FILE)

print(f"Training images has shape: {training_images.shape} and dtype: {training_images.dtype}")
print(f"Training labels has shape: {training_labels.shape} and dtype: {training_labels.dtype}")
print(f"Validation images has shape: {validation_images.shape} and dtype: {validation_images.dtype}")
print(f"Validation labels has shape: {validation_labels.shape} and dtype: {validation_labels.dtype}")

## 2.c Visualize the data

In [ ]:
def plot_categories(training_images, training_labels):
  fig, axes = plt.subplots(1, 10, figsize=(16, 15))
  axes = axes.flatten()
  letters = list(string.ascii_lowercase)

  for k in range(10):
    img = training_images[k]
    img = np.expand_dims(img, axis=-1)
    img = array_to_img(img)
    ax = axes[k]
    ax.imshow(img, cmap="Greys_r")
    ax.set_title(f"{letters[int(training_labels[k])]}")
    ax.set_axis_off()

  plt.tight_layout()
  plt.show()

plot_categories(training_images, training_labels)

## 2.d* preprocess data

## 2.e* use data augmentation and dropout

In [ ]:
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
  # add another dimension to the data
  # if your array is (10000, 28, 28)
  # You will need to make it (10000, 28, 28, 1)
  training_images = np.expand_dims(training_images,3)
  validation_images = np.expand_dims(validation_images,3)

  train_datagen = ImageDataGenerator(rescale=1/255.,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

  train_generator = train_datagen.flow(x=training_images,
                                       y=training_labels,
                                       batch_size=32) 

  validation_datagen = ImageDataGenerator(rescale=1/255.)

  validation_generator = validation_datagen.flow(x=validation_images,
                                                 y=validation_labels,
                                                 batch_size=32) 

  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)

print(f"Images of training generator have shape: {train_generator.x.shape}")
print(f"Labels of training generator have shape: {train_generator.y.shape}")
print(f"Images of validation generator have shape: {validation_generator.x.shape}")
print(f"Labels of validation generator have shape: {validation_generator.y.shape}")

# 3. Create, Train and Fit model

In [ ]:
from tensorflow.keras import optimizers, losses
def create_model():

    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation = "relu"),
        tf.keras.layers.Dense(26, activation = "softmax"),
  ])

    model.compile(loss=losses.SparseCategoricalCrossentropy(),
                  optimizer=optimizers.Adam(lr = 0.001),
                  metrics=['accuracy']) 
 
    return model

In [ ]:
model = create_model()

history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator)

## 3.a* use (pre-trained model)transfer learning if necessary

## 3.b* extract features of pre-trained model

## 3.c* match test data to inputshape of neural network

## 3.d* match output data of a neural network to speicifed inputshape for test data

## 3.e* use batch loading of data

## 3.f* use callbacks to trigger the end of training cycles

# 4. Evaluate the model

## 4.a Visualize the model

## 4.b* Plot loss and accuracy of a trained model

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## 4.c* use model to predict results